# 4. 予測関数定義
# 4. Prediction function definition


In [ ]:
# 必要ライブラリのインポート
# Import required library

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
# PyTorch関連ライブラリ
# Pytorch related libraries
import torch
import torch.nn as nn
from torchviz import make_dot

In [ ]:
# デフォルトフォントサイズ変更
# Change default font size
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
# Change default graph size
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
# Turn grid display on
plt.rcParams['axes.grid'] = True

# numpyの浮動小数点の表示精度
# floating point display precision of numpy
np.set_printoptions(suppress=True, precision=4)

## 4.4 擬似コードとしての予測関数
## 4.4 Prediction function as pseudocode

In [ ]:
# レイヤー関数定義
# Layer function definition

# 最初の線形関数
# First linear function
# Input numbers: 784
# Output numbers: 128
l1 = nn.Linear(784, 128)

# 2番目の線形関数
# Second linear function
# Input numbers: 128
# Output numbers: 10
l2 = nn.Linear(128, 10)

# 活性化関数
# Activation function
relu = nn.ReLU(inplace=True)

In [ ]:
# 入力テンソルから出力テンソルを計算
# Calculating output tensor from input tensor

# ダミー入力データを作成
# Creating dummy input data
inputs = torch.randn(100, 784)

# 中間テンソル1の計算
# Calculation of intermediate tensor 1
m1 = l1(inputs)

# 中間テンソル2の計算
# Calculation of intermediate tensor 2
m2 = relu(m1)

# 出力テンソルの計算
# Calculation of output tensor
outputs = l2(m2)

# 入力テンソルと出力テンソルのshape確認
# Chack shape of input and output tensor
print('入力テンソル', inputs.shape)
print('出力テンソル', outputs.shape)

In [ ]:
# nn.Sequentialを使って、全体を合成関数として定義
# Define the whole as a composite function using nn.Sequential

net2 = nn.Sequential(
    l1,
    relu,
    l2
)

outputs2 = net2(inputs)

# 入力テンソルと出力テンソルのshape確認
# Check shape of input and output tensor
print('入力テンソル', inputs.shape)
print('出力テンソル', outputs2.shape)

## 4.7　活性化関数の目的
## 4.7  The purpose of activation function

当コラムでは予測結果のグラフ(図4-9から図4-11) が重要で、そのための実装コードの意味が現段階でわからなくて構いません。

In this column, the graph of prediction result (from figure 4-9 to 4-11) is important, so it's OK you don't understand the meaning of code for now.

以下のコードがあくまで参考情報としての提示になります

The following code is provided for reference purpose only.

#### 学習用のデータの計算
#### Calculation of learning data

In [ ]:
# 訓練データ、検証データの計算
# Calculation of training and validation data
np.random.seed(123)
x = np.random.randn(100,1)

# yの値はx^2に乱数の要素を1/10程度付加した
# The value of y has added a random number factor of about 1/10 to x^2
y = x**2 + np.random.randn(100,1) * 0.1

# データを50件ずつに分け、それぞれ訓練用、検証用とする
# Divide the data into 50 sets, one for training and one for validation
x_train = x[:50,:]
x_test = x[50:,:]
y_train = y[:50,:]
y_test = y[50:,:]

In [ ]:
# 散布図表示
# Show scatter plot
plt.scatter(x_train, y_train, c='b', label='訓練データ')
plt.scatter(x_test, y_test, c='k', marker='x', label='検証データ')
plt.legend()
plt.show()

In [ ]:
# 入力変数x と正解値 ytのTesor化
# Convert input variable x and true value yt to tensor
inputs = torch.tensor(x_train).float()
labels = torch.tensor(y_train).float()

inputs_test = torch.tensor(x_test).float()
labels_test = torch.tensor(y_test).float()

In [ ]:
import torch.optim as optim

### 単回帰の場合
###  Simple regression

In [ ]:
# モデルの定義
# Definition of the model

class Net(nn.Module):
    def __init__(self):
        # 親クラスnn.Modulesの初期化呼び出し
        # Call initialization of parent class nn.Modules
        super().__init__()

        # 出力層の定義
        # Definition of output layer
        self.l1 = nn.Linear(1, 1)   
        
    # 予測関数の定義
    # Definition of prediction function
    def forward(self, x):
        x1 = self.l1(x) # 線形回帰 Linear regression
        return x1

In [ ]:
# 学習率
# Learning rate
lr = 0.01

# インスタンス生成　(パラメータ値初期化)
# Creating instance (Initializing parameter value)
net = Net()

# 最適化アルゴリズム: 勾配降下法
# Optimization algorithm: gradient decent method
optimizer = optim.SGD(net.parameters(), lr=lr)

# 損失関数： 最小二乗誤差
# Loss function: Least suqare error
criterion = nn.MSELoss()

# 繰り返し回数
# Repeat count
num_epochs = 10000

# 評価結果記録用 (損失関数値のみ記録)
# Validation results recording (recording only loss function values)
history = np.zeros((0,2))

In [ ]:
# 繰り返し計算メインループ
# Repeat calculation main loop

for epoch in range(num_epochs):
    
    # 勾配値初期化
    # Gradient value initialization
    optimizer.zero_grad()

    # 予測計算
    # Prediction calculation
    outputs = net(inputs)
  
    # 誤差計算
    # error calculation
    loss = criterion(outputs, labels)

    # 勾配計算
    # Gradient calculation
    loss.backward()

    # 勾配降下法の適用
    # Applying gradient descent
    optimizer.step()

    # 100回ごとに途中経過を記録する
    # Recording the progress every 100 times
    if ( epoch % 100 == 0):
        history = np.vstack((history, np.array([epoch, loss.item()])))
        print(f'Epoch {epoch} loss: {loss.item():.5f}')

In [ ]:
# 結果のグラフ化
# Show the result graph
labels_pred = net(inputs_test)

plt.title('隠れ層なし　活性化関数なし')
plt.scatter(inputs_test[:,0].data, labels_pred[:,0].data, c='b', label='予測値')
plt.scatter(inputs_test[:,0].data, labels_test[:,0].data, c='k', marker='x',label='正解値')
plt.legend()
plt.show()

### 疑似ディープラーニングの場合
### Pseudo deep learning

In [ ]:
# モデルの定義
# Definition of model

class Net2(nn.Module):
    def __init__(self):
        # 親クラスnn.Modulesの初期化呼び出し
        # Call initializing parent class nn.modules
        super().__init__()

        # 出力層の定義
        # Definition of output layer
        self.l1 = nn.Linear(1, 10)
        self.l2 = nn.Linear(10, 10)
        self.l3 = nn.Linear(10,1)
        
    # 予測関数の定義
    # Definition of prediction function
    def forward(self, x):
        x1 = self.l1(x)
        x2 = self.l2(x1)
        x3 = self.l3(x2)
        return x3

In [ ]:
# 学習率
# Learning rate

lr = 0.01

# インスタンス生成　(パラメータ値初期化)
# Creating instace (Initializing parameter value)
net2 = Net2()

# 最適化アルゴリズム: 勾配降下法
# Optimizaiotn algorithm: gardient descent method
optimizer = optim.SGD(net2.parameters(), lr=lr)

# 損失関数： 最小二乗誤差
# loss function: Least square error
criterion = nn.MSELoss()

# 繰り返し回数
# Repeat count
num_epochs = 10000

# 評価結果記録用 (損失関数値のみ記録)
# Validation result recording (Recording loss function value)
history = np.zeros((0,2))

In [ ]:
# 繰り返し計算メインループ
# Repeat calculation main loop
for epoch in range(num_epochs):
    
    # 勾配値初期化
    # Initializing gradient value
    optimizer.zero_grad()

    # 予測計算
    # Prediction calculation
    outputs = net2(inputs)
  
    # 誤差計算
    # error calculation
    loss = criterion(outputs, labels)

    # 勾配計算
    # gradient calculation
    loss.backward()

    # 勾配降下法の適用
    # Applying gradient descent method
    optimizer.step()

    # 100回ごとに途中経過を記録する
    # Recording the progress every 100 times
    if ( epoch % 100 == 0):
        history = np.vstack((history, np.array([epoch, loss.item()])))
        print(f'Epoch {epoch} loss: {loss.item():.5f}')

In [ ]:
# 結果のグラフ化
# Show result graph
labels_pred2 = net2(inputs_test)

plt.title('隠れ層２層　活性化関数なし')
plt.scatter(inputs_test[:,0].data, labels_pred2[:,0].data, c='b', label='予測値')
plt.scatter(inputs_test[:,0].data, labels_test[:,0].data, c='k', marker='x',label='正解値')
plt.legend()
plt.show()


### ディープラーニング(活性化関数あり)の場合
### Deep learning (with avtivation function)

In [ ]:
# モデルの定義
# Definition of model

class Net3(nn.Module):
    def __init__(self):
        # 親クラスnn.Modulesの初期化呼び出し
        # Call initialization of parent class nn.Modules
        super().__init__()

        # 出力層の定義
        # Definition of output layer
        self.l1 = nn.Linear(1, 10)
        self.l2 = nn.Linear(10, 10)
        self.l3 = nn.Linear(10,1)
        self.relu = nn.ReLU(inplace=True)
        
    # 予測関数の定義
    # Definition of prediction function
    def forward(self, x):
        x1 = self.relu(self.l1(x))
        x2 = self.relu(self.l2(x1))
        x3 = self.l3(x2)
        return x3

In [ ]:
# 学習率
# Learning rate
lr = 0.01

# インスタンス生成　(パラメータ値初期化)
# Creating instance (Initializing parameter value)
net3 = Net3()

# 最適化アルゴリズム: 勾配降下法
# Optimization algorithm: gradient descent method
optimizer = optim.SGD(net3.parameters(), lr=lr)

# 損失関数： 最小二乗誤差
# loss function: Least square error
criterion = nn.MSELoss()

# 繰り返し回数
# repeat count
num_epochs = 10000

# 評価結果記録用 (損失関数値のみ記録)
# Validation result recording (Recording only loss function value)
history = np.zeros((0,2))

In [ ]:
# 繰り返し計算メインループ
# Repeat calculation main loop

for epoch in range(num_epochs):
    
    # 勾配値初期化
    # Initializing gradient value
    optimizer.zero_grad()

    # 予測計算
    # Prediction calculation
    outputs = net3(inputs)
  
    # 誤差計算
    # Error calculation
    loss = criterion(outputs, labels)

    # 勾配計算
    # Gradient calculation
    loss.backward()

    # 勾配降下法の適用
    # Applying gradient descent method
    optimizer.step()

    # 100回ごとに途中経過を記録する
    # Recording the progress every 100 times
    if ( epoch % 100 == 0):
        history = np.vstack((history, np.array([epoch, loss.item()])))
        print(f'Epoch {epoch} loss: {loss.item():.5f}')

In [ ]:
# 結果の可視化
# Visualizing resutls
labels_pred3 = net3(inputs_test)

plt.title('隠れ層２層　活性化関数あり')
plt.scatter(inputs_test[:,0].data, labels_pred3[:,0].data, c='b', label='予測値')
plt.scatter(inputs_test[:,0].data, labels_test[:,0].data, c='k', marker='x',label='正解値')
plt.legend()
plt.show()
